In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
weights = np.array( [[1,2,3,4],[5,6,7,8],[9,10,11,12]], dtype=np.float32)
inputs = np.array([[0.1,0.2,0.3]], dtype=np.float32)
bias = np.array([-2.2, -3.33, -4.444, -5.5555], dtype=np.float32)
dots = np.dot(inputs,weights)
b_dots = dots + bias 
outs = np.tanh(b_dots)

print ("Weights:\n", weights)
print ("Inputs:\n", inputs)
print ("Bias: \n", bias)
print ("Dot Product:\n", dots)
print ("Biased Products:\n", b_dots)
print ("Output:\n", outs)

Weights:
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
Inputs:
 [[0.1 0.2 0.3]]
Bias: 
 [-2.2    -3.33   -4.444  -5.5555]
Dot Product:
 [[3.8000002 4.4       5.        5.6000004]]
Biased Products:
 [[1.6000001  1.0700002  0.55600023 0.04450035]]
Output:
 [[0.9216686  0.7894613  0.50500375 0.044471  ]]


In [11]:
import numpy as np
import struct

def flt_to_hex(f):
    return hex(struct.unpack('<I', struct.pack('<f', f))[0])

weights = np.array( [[1,2,3,4],[5,6,7,8],[9,10,11,12]], dtype=np.float32)
inputs = np.array([[0.1,0.2,0.3]], dtype=np.float32)
bias = np.array([-2.2, -3.33, -4.444, -5.5555], dtype=np.float32)
dots = np.dot(inputs,weights)[0]
b_dots = dots + bias 
outs = np.tanh(b_dots)

flts = bias
flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], flts))
flts_str = list(map('{:+2.9f}'.format, flts))
offset=6
print ('parameter NUM_VALUES = %d,' % len(flts))
print ('parameter [31:0] VALUES [0:NUM_VALUES-1] = {')
for i in range(0, len(flts), offset):
    print ('//' + ', '.join(flts_str[i:i+offset]), end='\n')
    print ('  ' + ', '.join(flts_hex[i:i+offset] ), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('}') 

parameter NUM_VALUES = 4,
parameter [31:0] VALUES [0:NUM_VALUES-1] = {
//-2.200000048, -3.329999924, -4.443999767, -5.555500031
  32'hc00ccccd, 32'hc0551eb8, 32'hc08e353f, 32'hc0b1c6a8 
}


In [10]:
import numpy as np
import struct

def flt_to_hex(f):
    return hex(struct.unpack('<I', struct.pack('<f', f))[0])

weights = np.array( [[1,2,3,4],[5,6,7,8],[9,10,11,12]], dtype=np.float32)
inputs = np.array([[0.1,0.2,0.3]], dtype=np.float32)
bias = np.array([-2.2, -3.33, -4.444, -5.5555], dtype=np.float32)
dots = np.dot(inputs,weights)
b_dots = dots + bias 
outs = np.tanh(b_dots)

flts = inputs[0]
flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], flts))
flts_str = list(map('"{:2.4f}"'.format, flts))
offset=6
print ('static bit [31:0] fpHex [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_hex[i:i+offset] ), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 
print ('static string fpAscii [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_str[i:i+offset]), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};')
print ('static int MAX_SIZE = %d;' % len(flts))  

static bit [31:0] fpHex [0:2] = {
 32'h3dcccccd, 32'h3e4ccccd, 32'h3e99999a 
};
static string fpAscii [0:2] = {
 "0.1000", "0.2000", "0.3000" 
};
static int MAX_SIZE = 3;


In [14]:
import numpy as np
import struct
LUT_LIM = 2 # seems to be flattened
LUT_SIZE = 64 # 64 yields 95.6% accuracy
LUT_STEP = ((LUT_LIM+LUT_LIM)/LUT_SIZE)
LUT_XS = ( np.arange(-LUT_LIM, LUT_LIM, step=LUT_STEP) + 
            np.arange(LUT_LIM, -LUT_LIM, step=-LUT_STEP)[::-1] ) /2
LUT_YS = np.array( list((map( lambda x: 
                            np.tanh(x), 
                            LUT_XS))), dtype=np.float32)
X_SHIFT = int(LUT_LIM * (2<<15))
X_MAX = int( LUT_XS[-1] * (2<<15)) + X_SHIFT
def vlog_tanh(x):
    x_int = int(x * (2<<15)) & (2**32-1)    
    x_scaled = (x_int + X_SHIFT) & (2**32-1)
    lut_idx = x_scaled >> 12    
    if (lut_idx & X_MAX): return LUT_YS[0]
    elif (lut_idx >= LUT_SIZE): return LUT_YS[-1]
    else:  return LUT_YS[lut_idx] 

def flt_to_hex(f):
    return hex(struct.unpack('<I', struct.pack('<f', f))[0]) 

weights = np.array( [[1,2,3,4],[5,6,7,8],[9,10,11,12]], dtype=np.float32)
inputs = np.array([[0.1,0.2,0.3]], dtype=np.float32)
bias = np.array([-2.2, -3.33, -4.444, -5.5555], dtype=np.float32)
dots = np.dot(inputs,weights)[0]
b_dots = dots + bias 
outs = list(map(vlog_tanh, b_dots))

flts = outs
flts_hex = list(map(lambda x: '32\'h' + flt_to_hex(x)[2:], flts))
flts_str = list(map('"{:2.4f}"'.format, flts))
offset=6
print ('static bit [31:0] fpHex [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_hex[i:i+offset] ), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 
print ('static string fpAscii [0:' + str(len(flts)-1) + '] = {')
for i in range(0, len(flts), offset):
    print (' ' + ', '.join(flts_str[i:i+offset]), end='')
    print (',' if i < len(flts) - offset else ' ')
print ('};') 
print ('static int MAX_SIZE = %d;' % len(flts))  

static bit [31:0] fpHex [0:3] = {
 32'h3f6bb475, 32'h3f4c59a3, 32'h3ef90109, 32'h3cffeaad 
};
static string fpAscii [0:3] = {
 "0.9207", "0.7982", "0.4863", "0.0312" 
};
static int MAX_SIZE = 4;
